In [16]:
import sqlite3

In [9]:
pip install "pandas<2.1.0"


  Using cached pandas-2.0.3-cp38-cp38-macosx_10_9_x86_64.whl.metadata (18 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.7/11.7 MB 20.6 MB/s eta 0:00:00 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [5]:
import pandas as pd

In [5]:
pip install nba_api

Note: you may need to restart the kernel to use updated packages.


In [17]:
conn = sqlite3.connect('nba_shots.db')

In [11]:
def create_table_from_dataframe(df, table_name, conn):
    cursor = conn.cursor()
    # Generar la sentencia SQL para crear la tabla
    columns = ', '.join([f"{col.replace(' ', '_').lower()} {get_sql_type(df[col])}" for col in df.columns])
    create_table_sql = f"CREATE TABLE IF NOT EXISTS {table_name} ({columns});"
    cursor.execute(create_table_sql)
    conn.commit()

In [18]:
from nba_api.stats.endpoints import shotchartdetail
from nba_api.stats.static import teams

Extract data from API

In [19]:
team_dict = teams.find_teams_by_full_name('Los Angeles Lakers')[0]
team_id = team_dict['id']
    
# Llamar a la API de NBA para obtener los detalles de los tiros
response = shotchartdetail.ShotChartDetail(
    team_id=team_id,
    player_id=0,  # 0 para todos los jugadores
    season_nullable='2023-24',
    context_measure_simple='FGA'
)
# Convertir los resultados a un DataFrame de pandas
df = pd.DataFrame(response)

ValueError: DataFrame constructor not properly called!

In [32]:
df.columns

Index(['GRID_TYPE', 'GAME_ID', 'GAME_EVENT_ID', 'PLAYER_ID', 'PLAYER_NAME',
       'TEAM_ID', 'TEAM_NAME', 'PERIOD', 'MINUTES_REMAINING',
       'SECONDS_REMAINING', 'EVENT_TYPE', 'ACTION_TYPE', 'SHOT_TYPE',
       'SHOT_ZONE_BASIC', 'SHOT_ZONE_AREA', 'SHOT_ZONE_RANGE', 'SHOT_DISTANCE',
       'LOC_X', 'LOC_Y', 'SHOT_ATTEMPTED_FLAG', 'SHOT_MADE_FLAG', 'GAME_DATE',
       'HTM', 'VTM'],
      dtype='object')

In [11]:
df

pandas.core.frame.DataFrame

Data types for each column

In [27]:
def get_sql_type(col):
    if pd.api.types.is_integer_dtype(col):
        return "INTEGER"
    elif pd.api.types.is_float_dtype(col):
        return "REAL"
    elif pd.api.types.is_bool_dtype(col):
        return "BOOLEAN"
    else:
        return "TEXT"

Creating the table in the database

In [20]:
cursor = conn.cursor()
# Generar la sentencia SQL para crear la tabla
columns = ', '.join([f"{col.replace(' ', '_').lower()} {get_sql_type(df[col])}" for col in df.columns])
create_table_sql = f"CREATE TABLE IF NOT EXISTS shots ({columns});"
cursor.execute(create_table_sql)
conn.commit()

TypeError: 'pandas._libs.properties.AxisProperty' object is not iterable

Insert data in the database

In [29]:
cursor = conn.cursor()
# Generar la sentencia SQL para la inserción de datos
values = ', '.join(['?'] * len(df.columns))
insert_sql = f"INSERT INTO shots VALUES ({values})"
cursor.executemany(insert_sql, df.values.tolist())
conn.commit()

In [19]:

def extract_and_load_data(team_name, season, conn, context_mesure):
    # Obtener el ID del equipo
    team_dict = teams.find_teams_by_full_name('Los Angeles Lakers')[0]
    team_id = team_dict['id']
    
    # Llamar a la API de NBA para obtener los detalles de los tiros
    response = shotchartdetail.ShotChartDetail(
        team_id=team_id,
        player_id=0,  # 0 para todos los jugadores
        season_nullable=season,
        context_measure_simple=context_mesure
    )
    # Convertir los resultados a un DataFrame de pandas
    df = response.get_data_frames()[0]

    # Crear la tabla basada en los campos del DataFrame
    create_table_from_dataframe(df, 'shots', conn)

    # Insertar los datos en la tabla
    # insert_data_from_dataframe(df, 'shots', conn)

    #print(f"Datos insertados en la tabla 'shots' para el equipo {team_name} en la temporada {season}.")

In [23]:
extract_and_load_data('Los Angeles Lakers','2023-24',conn)

In [30]:
cursor = conn.cursor()

# Execute a query to select all data from the 'shots' table
cursor.execute('SELECT * FROM shots LIMIT 10')  # You can change LIMIT to see more rows

# Fetch all results from the executed query
rows = cursor.fetchall()

# Display the results
for row in rows:
    print(row)

('Shot Chart Detail', '0022300015', 11, 2544, 'LeBron James', 1610612747, 'Los Angeles Lakers', 1, 11, 12, 'Made Shot', 'Running Layup Shot', '2PT Field Goal', 'Restricted Area', 'Center(C)', 'Less Than 8 ft.', 0, 2, 5, 1, 1, '20231110', 'PHX', 'LAL')
('Shot Chart Detail', '0022300015', 17, 1629629, 'Cam Reddish', 1610612747, 'Los Angeles Lakers', 1, 10, 24, 'Made Shot', 'Jump Shot', '3PT Field Goal', 'Right Corner 3', 'Right Side(R)', '24+ ft.', 23, 234, 46, 1, 1, '20231110', 'PHX', 'LAL')
('Shot Chart Detail', '0022300015', 27, 1626156, "D'Angelo Russell", 1610612747, 'Los Angeles Lakers', 1, 9, 44, 'Made Shot', 'Fadeaway Jump Shot', '2PT Field Goal', 'In The Paint (Non-RA)', 'Center(C)', 'Less Than 8 ft.', 5, 0, 58, 1, 1, '20231110', 'PHX', 'LAL')
('Shot Chart Detail', '0022300015', 36, 2544, 'LeBron James', 1610612747, 'Los Angeles Lakers', 1, 9, 0, 'Made Shot', 'Driving Finger Roll Layup Shot', '2PT Field Goal', 'Restricted Area', 'Center(C)', 'Less Than 8 ft.', 0, 1, 8, 1, 1, '20

In [21]:
response = teams.get_teams()

    # Convertir los resultados a un DataFrame de pandas
df = pd.DataFrame(response)

In [22]:
df

,id,full_name,abbreviation,nickname,city,state,year_founded
0,1610612737,Atlanta Hawks,ATL,Hawks,Atlanta,Georgia,1949
1,1610612738,Boston Celtics,BOS,Celtics,Boston,Massachusetts,1946
2,1610612739,Cleveland Cavaliers,CLE,Cavaliers,Cleveland,Ohio,1970
3,1610612740,New Orleans Pelicans,NOP,Pelicans,New Orleans,Louisiana,2002
4,1610612741,Chicago Bulls,CHI,Bulls,Chicago,Illinois,1966
5,1610612742,Dallas Mavericks,DAL,Mavericks,Dallas,Texas,1980
6,1610612743,Denver Nuggets,DEN,Nuggets,Denver,Colorado,1976
7,1610612744,Golden State Warriors,GSW,Warriors,Golden State,California,1946
8,1610612745,Houston Rockets,HOU,Rockets,Houston,Texas,1967
9,1610612746,Los Angeles Clippers,LAC,Clippers,Los Angeles,California,1970


In [25]:
from nba_api.stats.endpoints import shotchartdetail, leaguegamelog, commonplayerinfo
from nba_api.stats.static import teams, players

# NBA API CALL
response = players.get_players()

# Convertir los resultados a un DataFrame de pandas
player_ids = pd.DataFrame(response)

all_players=[]

for player_id in player_ids['id']:
    response = commonplayerinfo.CommonPlayerInfo(player_id=player_id)
    player_info = response.get_data_frames()[0]
    all_players.append(player_info)

df=pd.concat(all_players,ignore_index=True)
df

ReadTimeout: HTTPSConnectionPool(host='stats.nba.com', port=443): Read timed out. (read timeout=30)

In [27]:

from nba_api.stats.endpoints import shotchartdetail, leaguegamelog, commonplayerinfo,leaguedashplayerbiostats
from nba_api.stats.static import teams, players

response = leaguedashplayerbiostats.LeagueDashPlayerBioStats(season='2023-24')
df = response.get_data_frames()[0]

df

,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_ABBREVIATION,AGE,PLAYER_HEIGHT,PLAYER_HEIGHT_INCHES,PLAYER_WEIGHT,COLLEGE,COUNTRY,...,GP,PTS,REB,AST,NET_RATING,OREB_PCT,DREB_PCT,USG_PCT,TS_PCT,AST_PCT
0,1631260,AJ Green,1610612749,MIL,24.0,6-4,76,190,Northern Iowa,USA,...,56,252,64,30,3.5,0.015,0.089,0.150,0.617,0.068
1,1631100,AJ Griffin,1610612737,ATL,20.0,6-6,78,220,Duke,USA,...,20,48,18,5,-14.0,0.011,0.108,0.167,0.382,0.041
2,1630639,AJ Lawson,1610612742,DAL,23.0,6-6,78,179,South Carolina,Canada,...,42,136,50,20,1.2,0.039,0.105,0.180,0.519,0.089
3,203932,Aaron Gordon,1610612743,DEN,28.0,6-8,80,235,Arizona,USA,...,73,1013,471,259,8.7,0.078,0.129,0.174,0.607,0.147
4,1628988,Aaron Holiday,1610612745,HOU,27.0,6-0,72,185,UCLA,USA,...,78,514,123,140,2.9,0.017,0.075,0.158,0.578,0.157
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
567,203897,Zach LaVine,1610612741,CHI,29.0,6-5,77,200,UCLA,USA,...,25,487,129,98,-5.4,0.009,0.134,0.238,0.578,0.172
568,1630285,Zavier Simpson,1610612763,MEM,27.0,6-0,72,190,Michigan,USA,...,7,42,20,25,-8.6,0.020,0.102,0.161,0.377,0.231
569,1630192,Zeke Nnaji,1610612743,DEN,23.0,6-9,81,240,Arizona,USA,...,58,186,125,32,-11.9,0.099,0.103,0.150,0.528,0.080
570,1630533,Ziaire Williams,1610612763,MEM,22.0,6-9,81,185,Stanford,USA,...,51,420,180,75,-9.9,0.031,0.138,0.192,0.511,0.118


In [36]:
cursor = conn.cursor()
primary_keys=['GAME_ID','TEAM_ID','PLAYER_ID']
# Create SQL command to create table with primary keys
columns = ', '.join([f"{col.replace(' ', '_').lower()} {get_sql_type(df[col])}" for col in df.columns])
primary_keys_str = ', '.join([key.replace(' ', '_').lower() for key in primary_keys])
create_table_sql = f"""
CREATE TABLE IF NOT EXISTS test_shots (
    {columns},
    PRIMARY KEY ({primary_keys_str})
)
"""
cursor.execute(create_table_sql)
conn.commit()

In [30]:



def get_sql_type(col):
    if pd.api.types.is_integer_dtype(col):
        return "INTEGER"
    elif pd.api.types.is_float_dtype(col):
        return "REAL"
    elif pd.api.types.is_bool_dtype(col):
        return "BOOLEAN"
    else:
        return "TEXT"


def insert_data_from_dataframe(df, table_name, conn):
    cursor = conn.cursor()
    # Generar la sentencia SQL para la inserción de datos
    placeholders = ', '.join(['?'] * len(df.columns))
    columns= ', '.join(df.columns)
    insert_sql = f"INSERT OR IGNORE INTO {table_name} ({columns}) VALUES ({placeholders})"
    cursor.executemany(insert_sql, df.values.tolist())
    conn.commit()

def extract_and_load_data_team_shots(team_name, table_name, season, conn, context_measure, primary_keys):
    # Obtener el ID del equipo
    team_dict = teams.find_teams_by_full_name(team_name)[0]
    team_id = team_dict['id']
    
    # Llamar a la API de NBA para obtener los detalles de los tiros
    response = shotchartdetail.ShotChartDetail(
        team_id=team_id,
        player_id=0,  # 0 para todos los jugadores
        season_nullable=season,
        context_measure_simple=context_measure
    )

    # Convertir los resultados a un DataFrame de pandas
    df = response.get_data_frames()[0]

    missing_keys = [key for key in primary_keys if key not in df.columns]
    if missing_keys:
        raise ValueError(f"The following primary key columns are missing from the DataFrame: {missing_keys}")


    create_table_from_dataframe(df, table_name , conn, primary_keys)

    # Insertar los datos en la tabla
    insert_data_from_dataframe(df, table_name, conn)

    print(f"Data has been inserted into the {table_name} table for the team {team_name} in the {season} season.")

In [ ]:
# Crear la tabla basada en los campos del DataFrame
    create_table_from_dataframe(df, table_name , conn, primary_keys)

    # Insertar los datos en la tabla
    insert_data_from_dataframe(df, table_name, conn)

    print(f"Data has been inserted into the {table_name} table for the team {team_name} in the {season} season.")

In [37]:
from nba_api.stats.endpoints import shotchartdetail, leaguegamelog, commonplayerinfo,leaguedashplayerbiostats
from nba_api.stats.static import teams, players
import pandas as pd
team_name='Los Angeles Lakers'
season='2023-24'
context_measure='FGA'
primary_keys=['GAME_ID','TEAM_ID','PLAYER_ID']

# Obtener el ID del equipo
team_dict = teams.find_teams_by_full_name(team_name)[0]
team_id = team_dict['id']

# Llamar a la API de NBA para obtener los detalles de los tiros
response = shotchartdetail.ShotChartDetail(
    team_id=0,
    player_id=0,  # 0 para todos los jugadores
    season_nullable=season,
    context_measure_simple=context_measure
)

# Convertir los resultados a un DataFrame de pandas
df = response.get_data_frames()[0]

missing_keys = [key for key in primary_keys if key not in df.columns]
if missing_keys:
    raise ValueError(f"The following primary key columns are missing from the DataFrame: {missing_keys}")


df

,GRID_TYPE,GAME_ID,GAME_EVENT_ID,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_NAME,PERIOD,MINUTES_REMAINING,SECONDS_REMAINING,...,SHOT_ZONE_AREA,SHOT_ZONE_RANGE,SHOT_DISTANCE,LOC_X,LOC_Y,SHOT_ATTEMPTED_FLAG,SHOT_MADE_FLAG,GAME_DATE,HTM,VTM
0,Shot Chart Detail,0022300001,7,1626167,Myles Turner,1610612754,Indiana Pacers,1,11,41,...,Center(C),Less Than 8 ft.,2,2,21,1,1,20231103,IND,CLE
1,Shot Chart Detail,0022300001,9,1630596,Evan Mobley,1610612739,Cleveland Cavaliers,1,11,23,...,Center(C),Less Than 8 ft.,7,59,53,1,0,20231103,IND,CLE
2,Shot Chart Detail,0022300001,11,1628971,Bruce Brown,1610612754,Indiana Pacers,1,11,15,...,Center(C),Less Than 8 ft.,2,13,18,1,0,20231103,IND,CLE
3,Shot Chart Detail,0022300001,13,1631097,Bennedict Mathurin,1610612754,Indiana Pacers,1,11,9,...,Right Side Center(RC),24+ ft.,28,151,237,1,1,20231103,IND,CLE
4,Shot Chart Detail,0022300001,15,1628378,Donovan Mitchell,1610612739,Cleveland Cavaliers,1,10,56,...,Center(C),Less Than 8 ft.,2,-20,1,1,1,20231103,IND,CLE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
218696,Shot Chart Detail,0022301230,644,1641720,Jalen Hood-Schifino,1610612747,Los Angeles Lakers,4,1,39,...,Center(C),16-24 ft.,22,64,218,1,0,20231207,LAL,NOP
218697,Shot Chart Detail,0022301230,646,1641722,Jordan Hawkins,1610612740,New Orleans Pelicans,4,1,31,...,Left Side(L),16-24 ft.,21,-206,71,1,0,20231207,LAL,NOP
218698,Shot Chart Detail,0022301230,648,1631108,Max Christie,1610612747,Los Angeles Lakers,4,1,13,...,Center(C),8-16 ft.,11,-15,115,1,1,20231207,LAL,NOP
218699,Shot Chart Detail,0022301230,649,1641722,Jordan Hawkins,1610612740,New Orleans Pelicans,4,0,59,...,Center(C),16-24 ft.,18,-32,179,1,0,20231207,LAL,NOP
